# Dėmesio mechanizmai ir transformatoriai

Viena iš pagrindinių pasikartojančių tinklų trūkumų yra ta, kad visi žodžiai sekoje turi vienodą įtaką rezultatui. Tai lemia neoptimalų standartinių LSTM koduotojo-dekoduotojo modelių veikimą atliekant sekos į seką užduotis, tokias kaip pavadintų objektų atpažinimas ar mašininis vertimas. Iš tiesų, tam tikri žodžiai įvesties sekoje dažnai turi didesnę įtaką išvesties sekai nei kiti.

Apsvarstykime sekos į seką modelį, pavyzdžiui, mašininį vertimą. Jis įgyvendinamas naudojant du pasikartojančius tinklus, kur vienas tinklas (**koduotojas**) suspaudžia įvesties seką į paslėptą būseną, o kitas tinklas (**dekoduotojas**) išskleidžia šią paslėptą būseną į išverstą rezultatą. Problema su šiuo požiūriu yra ta, kad tinklo galutinė būsena sunkiai prisimena sakinio pradžią, todėl modelis prastai veikia su ilgais sakiniais.

**Dėmesio mechanizmai** suteikia galimybę įvertinti kiekvieno įvesties vektoriaus kontekstinę įtaką kiekvienai RNN išvesties prognozei. Tai įgyvendinama sukuriant trumpesnius ryšius tarp tarpinių įvesties RNN būsenų ir išvesties RNN. Tokiu būdu, generuojant išvesties simbolį $y_t$, atsižvelgiama į visas įvesties paslėptas būsenas $h_i$, su skirtingais svorio koeficientais $\alpha_{t,i}$. 

![Vaizdas, rodantis koduotojo/dekoduotojo modelį su adityviniu dėmesio sluoksniu](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Koduotojo-dekoduotojo modelis su adityviniu dėmesio mechanizmu iš [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cituota iš [šio tinklaraščio įrašo](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Dėmesio matrica $\{\alpha_{i,j}\}$ atspindi, kokiu mastu tam tikri įvesties žodžiai prisideda prie tam tikro žodžio generavimo išvesties sekoje. Žemiau pateiktas tokios matricos pavyzdys:

![Vaizdas, rodantis pavyzdinį suderinimą, rastą naudojant RNNsearch-50, paimta iš Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Paveikslas paimtas iš [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (3 pav.)*

Dėmesio mechanizmai yra atsakingi už dabartinę arba beveik dabartinę pažangiausią natūralios kalbos apdorojimo būklę. Tačiau dėmesio pridėjimas žymiai padidina modelio parametrų skaičių, o tai sukėlė mastelio problemas su RNN. Vienas iš pagrindinių RNN mastelio apribojimų yra tas, kad modelių pasikartojantis pobūdis apsunkina mokymo partijų kūrimą ir lygiagretinimą. RNN kiekvienas sekos elementas turi būti apdorojamas nuosekliai, todėl jų lygiagretinimas yra sudėtingas.

Dėmesio mechanizmų pritaikymas kartu su šiuo apribojimu paskatino sukurti dabartinius pažangiausius transformatorių modelius, kuriuos šiandien naudojame, tokius kaip BERT ar OpenGPT3.

## Transformatorių modeliai

Užuot perdavę kiekvienos ankstesnės prognozės kontekstą į kitą vertinimo žingsnį, **transformatorių modeliai** naudoja **pozicinius kodavimus** ir **dėmesį**, kad užfiksuotų įvesties kontekstą tam tikrame teksto lange. Žemiau pateiktas paveikslas rodo, kaip poziciniai kodavimai kartu su dėmesiu gali užfiksuoti kontekstą tam tikrame lange.

![Animuotas GIF, rodantis, kaip atliekami vertinimai transformatorių modeliuose.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Kadangi kiekviena įvesties pozicija yra nepriklausomai susieta su kiekviena išvesties pozicija, transformatoriai gali geriau lygiagretinti nei RNN, o tai leidžia kurti daug didesnius ir išraiškingesnius kalbos modelius. Kiekviena dėmesio galvutė gali būti naudojama mokytis skirtingų žodžių tarpusavio ryšių, kurie pagerina natūralios kalbos apdorojimo užduotis.

## Paprasto transformatoriaus modelio kūrimas

Keras neturi įmontuoto transformatoriaus sluoksnio, tačiau mes galime sukurti savo. Kaip ir anksčiau, sutelksime dėmesį į AG News duomenų rinkinio teksto klasifikavimą, tačiau verta paminėti, kad transformatorių modeliai geriausius rezultatus pasiekia sudėtingesnėse NLP užduotyse.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nauji sluoksniai Keras turėtų paveldėti `Layer` klasę ir įgyvendinti `call` metodą. Pradėkime nuo **Pozicinio Įterpimo** sluoksnio. Naudosime [šiek tiek kodo iš oficialios Keras dokumentacijos](https://keras.io/examples/nlp/text_classification_with_transformer/). Mes darysime prielaidą, kad visi įvesties sekos yra užpildytos iki ilgio `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Šis sluoksnis susideda iš dviejų `Embedding` sluoksnių: vienas skirtas žodžių įterpimui (kaip aptarėme anksčiau), o kitas – pozicijų įterpimui. Pozicijos sukuriamos kaip natūralių skaičių seka nuo 0 iki `maxlen`, naudojant `tf.range`, ir tada perduodamos per įterpimo sluoksnį. Du gauti įterpimo vektoriai yra sudedami, sukuriant poziciškai įterptą įvesties reprezentaciją, kurios forma yra `maxlen`$\times$`embed_dim`.

Dabar įgyvendinkime transformatoriaus bloką. Jis priims anksčiau apibrėžto įterpimo sluoksnio išvestį:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Dabar esame pasiruošę apibrėžti pilną transformatoriaus modelį:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT transformerių modeliai

**BERT** (Bidirectional Encoder Representations from Transformers) yra labai didelis daugiasluoksnis transformerių tinklas, turintis 12 sluoksnių *BERT-base* versijoje ir 24 sluoksnius *BERT-large* versijoje. Modelis pirmiausia yra iš anksto apmokomas naudojant didelį tekstinių duomenų korpusą (Vikipedija + knygos) taikant nesupervizuotą mokymą (prognozuojant užmaskuotus žodžius sakinyje). Per šį išankstinį mokymą modelis įgyja reikšmingą kalbos supratimo lygį, kurį vėliau galima pritaikyti su kitais duomenų rinkiniais naudojant smulkųjį derinimą. Šis procesas vadinamas **perkėlimo mokymusi**.

![paveikslėlis iš http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Yra daug transformerių architektūrų variantų, įskaitant BERT, DistilBERT, BigBird, OpenGPT3 ir kitus, kuriuos galima smulkiai derinti.

Pažiūrėkime, kaip galime naudoti iš anksto apmokytą BERT modelį, kad išspręstume tradicinę sekų klasifikavimo problemą. Pasiskolinsime idėją ir šiek tiek kodo iš [oficialios dokumentacijos](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Norėdami įkelti iš anksto apmokytus modelius, naudosime **Tensorflow hub**. Pirmiausia įkelkime BERT specifinį vektorizatorių:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Svarbu naudoti tą patį vektorizatorių, kuris buvo naudojamas originaliam tinklui treniruoti. Be to, BERT vektorizatorius grąžina tris komponentus:
* `input_word_ids`, tai yra įvesties sakinio žodžių numerių seka
* `input_mask`, rodanti, kuri seka yra tikroji įvestis, o kuri – užpildas. Tai panašu į kaukę, kurią sukuria `Masking` sluoksnis
* `input_type_ids` naudojamas kalbos modeliavimo užduotims ir leidžia nurodyti du įvesties sakinius vienoje sekoje.

Tuomet galime sukurti BERT funkcijų ištraukiklį:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Taigi, BERT sluoksnis grąžina keletą naudingų rezultatų:
* `pooled_output` yra sekos visų žetonų vidurkis. Galite tai laikyti kaip išmanų viso tinklo semantinį įterpimą. Tai atitinka `GlobalAveragePooling1D` sluoksnio išvestį mūsų ankstesniame modelyje.
* `sequence_output` yra paskutinio transformatoriaus sluoksnio išvestis (atitinka `TransformerBlock` išvestį mūsų aukščiau esančiame modelyje).
* `encoder_outputs` yra visų transformatorių sluoksnių išvestys. Kadangi įkėlėme 4 sluoksnių BERT modelį (kaip tikriausiai galite nuspėti iš pavadinimo, kuriame yra `4_H`), jis turi 4 tensorius. Paskutinis iš jų yra toks pat kaip `sequence_output`.

Dabar apibrėšime viso proceso klasifikavimo modelį. Naudosime *funkcinį modelio apibrėžimą*, kai apibrėžiame modelio įvestį ir tada pateikiame seriją išraiškų, kad apskaičiuotume jo išvestį. Taip pat padarysime, kad BERT modelio svoriai nebūtų treniruojami, ir treniruosime tik galutinį klasifikatorių:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Nepaisant to, kad treniruojamų parametrų yra nedaug, procesas vyksta gana lėtai, nes BERT požymių ištraukiklis yra skaičiavimo požiūriu sudėtingas. Panašu, kad mums nepavyko pasiekti tinkamo tikslumo, galbūt dėl nepakankamo treniravimo arba dėl modelio parametrų trūkumo.

Pabandykime atšildyti BERT svorius ir taip pat jį treniruoti. Tam reikės labai mažo mokymosi greičio, taip pat atsargesnės treniravimo strategijos su **apšilimu**, naudojant **AdamW** optimizatorių. Naudosime `tf-models-official` paketą optimizatoriui sukurti:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Kaip matote, mokymas vyksta gana lėtai – tačiau galite eksperimentuoti ir treniruoti modelį kelis epochus (5–10), kad pamatytumėte, ar galite pasiekti geriausią rezultatą, palyginti su anksčiau naudotais metodais.

## Huggingface Transformers biblioteka

Kitas labai dažnas (ir šiek tiek paprastesnis) būdas naudoti Transformer modelius yra [HuggingFace paketas](https://github.com/huggingface/), kuris suteikia paprastus komponentus įvairioms NLP užduotims. Jis prieinamas tiek Tensorflow, tiek PyTorch – dar vienai labai populiariai neuroninių tinklų sistemai.

> **Note**: Jei nesate suinteresuoti pamatyti, kaip veikia Transformers biblioteka – galite praleisti šio užrašų knygelės pabaigą, nes nieko iš esmės naujo, palyginti su tuo, ką darėme aukščiau, nepamatysite. Mes kartosime tuos pačius BERT modelio mokymo žingsnius, naudodami kitą biblioteką ir žymiai didesnį modelį. Taigi procesas apima gana ilgą mokymą, todėl galite tiesiog peržvelgti kodą.

Pažiūrėkime, kaip mūsų problemą galima išspręsti naudojant [Huggingface Transformers](http://huggingface.co).


Pirmiausia turime pasirinkti modelį, kurį naudosime. Be kelių įmontuotų modelių, Huggingface turi [internetinį modelių saugyklą](https://huggingface.co/models), kur bendruomenė dalijasi daugybe iš anksto apmokytų modelių. Visi šie modeliai gali būti įkelti ir naudojami tiesiog nurodant modelio pavadinimą. Visi reikalingi dvejetainiai failai modeliui bus automatiškai atsisiųsti.

Tam tikrais atvejais jums gali prireikti įkelti savo modelius. Tokiu atveju galite nurodyti katalogą, kuriame yra visi susiję failai, įskaitant parametrus, skirtus tokenizeriui, `config.json` failą su modelio parametrais, dvejetainius svorius ir pan.

Iš modelio pavadinimo galime sukurti tiek modelį, tiek tokenizerį. Pradėkime nuo tokenizerio:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objektas turi `encode` funkciją, kuri gali būti tiesiogiai naudojama tekstui užkoduoti:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Mes taip pat galime naudoti tokenizatorių, kad užkoduotume seką taip, kad ji būtų tinkama perduoti modeliui, t. y. įtraukiant `token_ids`, `input_mask` laukus ir kt. Taip pat galime nurodyti, kad norime Tensorflow tensorių, pateikdami argumentą `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Šiuo atveju naudosime iš anksto apmokytą BERT modelį, vadinamą `bert-base-uncased`. *Uncased* reiškia, kad modelis yra nejautrus raidžių dydžiui.

Treniruojant modelį, turime pateikti tokenizuotą seką kaip įvestį, todėl sukursime duomenų apdorojimo procesą. Kadangi `tokenizer.encode` yra Python funkcija, naudosime tą patį metodą kaip ir paskutiniame skyriuje, iškviesdami ją naudojant `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Dabar galime įkelti faktinį modelį naudodami `BertForSequenceClassification` paketą. Tai užtikrina, kad mūsų modelis jau turi reikiamą klasifikavimo architektūrą, įskaitant galutinį klasifikatorių. Pamatysite įspėjimo pranešimą, kad galutinio klasifikatoriaus svoriai nėra inicializuoti, ir modelis reikalautų išankstinio mokymo - tai visiškai normalu, nes būtent tai mes ketiname daryti!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Kaip matote iš `summary()`, modelis turi beveik 110 milijonų parametrų! Tikėtina, kad jei norime paprastos klasifikavimo užduoties su palyginti mažu duomenų rinkiniu, nenorime treniruoti BERT bazinio sluoksnio:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Dabar esame pasiruošę pradėti mokymus!

> **Pastaba**: Pilno masto BERT modelio mokymas gali užtrukti labai daug laiko! Todėl mes jį treniruosime tik pirmosioms 32 partijoms. Tai tik tam, kad parodytume, kaip nustatomas modelio mokymas. Jei norite išbandyti pilno masto mokymą, tiesiog pašalinkite `steps_per_epoch` ir `validation_steps` parametrus ir pasiruoškite laukti!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Jei padidinsite iteracijų skaičių, palauksite pakankamai ilgai ir treniruositės kelis epochus, galite tikėtis, kad BERT klasifikacija suteiks geriausią tikslumą! Taip yra todėl, kad BERT jau gana gerai supranta kalbos struktūrą, ir mums tereikia pritaikyti galutinį klasifikatorių. Tačiau, kadangi BERT yra didelis modelis, visas treniravimo procesas užtrunka ilgai ir reikalauja rimtų skaičiavimo išteklių! (GPU, ir pageidautina daugiau nei vieno).

> **Pastaba:** Mūsų pavyzdyje naudojome vieną iš mažiausių iš anksto apmokytų BERT modelių. Yra didesnių modelių, kurie tikriausiai duotų geresnius rezultatus.


## Pagrindinės mintys

Šiame skyriuje aptarėme naujausias modelių architektūras, pagrįstas **transformeriais**. Mes pritaikėme jas savo teksto klasifikavimo užduočiai, tačiau BERT modeliai taip pat gali būti naudojami entitetų išskyrimui, klausimų-atsakymų sistemoms ir kitoms NLP užduotims.

Transformeriai šiuo metu yra pažangiausia technologija NLP srityje, ir daugeliu atvejų tai turėtų būti pirmasis sprendimas, kurį pradėsite išbandyti, kurdami individualius NLP sprendimus. Tačiau labai svarbu suprasti pagrindinius pasikartojančių neuroninių tinklų principus, aptartus šiame modulyje, jei norite kurti pažangius neuroninius modelius.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius naudojant šį vertimą.
